In [1]:
import os
import pandas as pd
import numpy as np
import torch
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification, AdamW

cuda


In [2]:
# define a class for the AMP data that will correctly format the sequence information
# for fine-tuning with huggingface API

class amp_data():
    def __init__(self, df, tokenizer_name='Rostlab/prot_bert_bfd', max_len=200):
        
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        self.max_len = max_len
        
        self.seqs, self.labels = self.get_seqs_labels()
        
    def get_seqs_labels(self):        
        # isolate the amino acid sequences and their respective AMP labels
        seqs = list(df['aa_seq'])
        labels = list(df['AMP'].astype(int))
        
#         assert len(seqs) == len(labels)
        return seqs, labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        seq = " ".join("".join(self.seqs[idx].split()))
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_len)
        
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])

        return sample

In [3]:
# read in the train dataset
# create an amp_data class of the dataset

df = pd.read_csv('/home/hansol/amp/all_veltri.csv', index_col = 0)
df = df.sample(frac=1, random_state = 0)
print(df.head(7))

train_dataset = amp_data(df)

                                                            aa_seq  aa_len  \
AP02151          YEALVTSILGKLTGLWHNDSVDFMGHICYFRRRPKIRRFKLYHEGK...      95   
AP01951                                          FLPLVLGALSGILPKIL      17   
AP00972                                        FLSLIPHAINAVGVHAKHF      19   
AP01261                                           IIEKLVNTALGLLSGL      16   
AP01298                                       GLFTLIKCAYQLIAPTVACN      20   
AP01802                                     RPWAGNGSVHRYTVLSPRLKTQ      22   
UniRef50_Q9UTR1                                SKENSYVEKLLYKQRFYAS      19   

                   AMP  
AP02151           True  
AP01951           True  
AP00972           True  
AP01261           True  
AP01298           True  
AP01802           True  
UniRef50_Q9UTR1  False  


In [4]:
# define the necessary metrics for performance evaluation

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
#     conf = confusion_matrix(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
#         'confusion matrix': conf
    }

In [5]:
# define the initializing function for Trainer in huggingface

def model_init():
    return AutoModelForSequenceClassification.from_pretrained('Rostlab/prot_bert_bfd')

In [7]:
# training on entire data
# no evaluation/validation

training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=15,              
    learning_rate=5e-5,              
    per_device_train_batch_size=1,   
    warmup_steps=0,               
    weight_decay=0.1,               
    logging_dir='./logs',            
    logging_steps=100,               
    do_train=True,                   
    do_eval=True,                   
    evaluation_strategy="no",    
    save_strategy='no',
    gradient_accumulation_steps=64,  
    fp16=True,                       
    fp16_opt_level="O2",             
    run_name="AMP-BERT",             
    seed=0,                          
    load_best_model_at_end = True
)

trainer = Trainer(
    model_init=model_init,                
    args=training_args,                   
    train_dataset=train_dataset,          
    compute_metrics = compute_metrics,    
)

trainer.train()
# trainer.save_model('/home/hansol/amp/model/')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/Rostlab/prot_bert_bfd/resolve/main/config.json from cache at /home/hansol/.cache/huggingface/transformers/67f460acc7e7e147ff828e909ffe419d00d66ce679c682bc4ab715c107bcbe41.baf557855a8618d0ddfb6c23bfd135bfc38ccf8c3fb099b8df45eb110ccf05e9
Model config BertConfig {
  "_name_or_path": "Rostlab/prot_bert_bfd",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 40000,
  "model_type": "bert",
  "num_at

Step,Training Loss
100,0.707100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=105, training_loss=0.6926387627919515, metrics={'train_runtime': 21761.5943, 'train_samples_per_second': 2.451, 'train_steps_per_second': 0.005, 'total_flos': 2.4064232304672e+16, 'train_loss': 0.6926387627919515, 'epoch': 14.88})

In [8]:
# performance metrics on the training data itself

predictions, label_ids, metrics = trainer.predict(train_dataset)
metrics

***** Running Prediction *****
  Num examples = 3556
  Batch size = 56


{'test_loss': 0.3816959261894226,
 'test_accuracy': 0.9569741282339708,
 'test_f1': 0.9561478933791918,
 'test_precision': 0.9748684979544127,
 'test_recall': 0.9381327334083239,
 'test_runtime': 132.835,
 'test_samples_per_second': 26.77,
 'test_steps_per_second': 0.482}